In [1]:
import numpy as np
import os,pickle,time
import matplotlib.pyplot as plt
from skimage import io
from skimage import img_as_float
from multislice import prop,prop_utils

In [2]:
def find_edge(x):
    if x<7500:
        if x>100:
            return 100
        else :
            return int(np.floor(x / 2) * 2)
    else :
        if (15000-x)>100:
            return 100
        else :
            return int(np.floor(15000-x / 2) * 2)

In [3]:
def get_focal_spot(focal_plane_):
    x_,y_ = np.where(focal_plane_==np.max(focal_plane_))
    x_ = x_[0]
    y_ = y_[0]
    print(x_,y_)
    x1 = find_edge(x_)
    y1 = find_edge(y_)
    focal_spot_ = np.zeros((200,200))
    if (x1+y1) != 200:
        focal_spot_[100-x1:100+x1,100-y1:100+y1] = focal_plane_[x_-x1:x_+x1,y_-y1:y_+y1]
    else :
        focal_spot_[:,:] = focal_plane_[x_-100:x_+100,y_-100:y_+100]
    return focal_spot_,x_,y_,np.max(focal_plane_)

In [4]:
def make_zp_from_rings(n,grid_size):
    os.chdir('/raid/home/sajid/ZP_simulations/var_width_thickness/4kev/rings')
    zp = np.zeros((grid_size,grid_size))
    for i in range(n):
        if i%2 == 1 :
            ring_ = np.load('ring_'+str(i)+'.npy')
            ring_ = tuple((ring_[0],ring_[1]))
            zp[ring_] = 1
    return zp

In [5]:
def tilt(i,zp,thickness,parameters,out_dir):
    zp_coords = parameters['zp_coords']    
    step_xy = parameters['step_xy']
    energy = parameters['energy(in eV)']
    wavel = parameters['wavelength in m']
    f = parameters['focal_length']
    L = step_xy*np.shape(zp)[0] 
    n = np.shape(zp)[0]         
    zp_thickness = thickness 
    
    os.chdir(out_dir)
    
    t1 = time.time()
    print('claclulating for tilt angle : ',i)      
    theta = (i)*(np.pi/180)
    slope = np.tan(theta)
    x = np.linspace(zp_coords[0]*1e-6,zp_coords[1]*1e-6,n)
    X,Y = np.meshgrid(x,x)
    z1 = 2*np.pi*(1/wavel)*slope*X
    wave_in = np.multiply(np.ones((n,n),dtype='complex64'),np.exp(1j*(z1)))
    
    zp_beta  = parameters['beta']*zp
    zp_delta = parameters['delta']*zp
    print(step_xy,wavel,zp_thickness)
    number_of_steps_zp =  prop_utils.number_of_steps(step_xy,wavel,zp_thickness)*2
    
    wave_focus = prop_utils.propogate_through_object(wave_in,zp_delta,zp_beta,zp_thickness,step_xy,wavel,L,
                                                     number_of_steps_zp,d1=0,d2=f, xray_object='zone Plate')
    focal_spot,x_,y_,max_val = get_focal_spot(np.abs(wave_focus))
    io.imsave('tilt_'+str(i)+'.tiff',img_as_float(focal_spot))
    t2 = time.time()
    print('tilt image number :',i,'time taken :',(t2-t1))
    print(x_,y_,max_val)
    return {'tilt_angle':i,'max_loc':np.array([x_,y_]),'max_val':max_val}

In [6]:
num_zones = np.array([50,100,200,300,400,500,600])
thickness = np.array([0.5e-6,1e-6,2e-6,4e-6,8e-6,10e-6])

In [7]:
inputs = np.linspace(-1,1,50)
parameters = pickle.load(open('parameters.pickle','rb'))
grid_size = parameters['grid_size']
print(parameters)

{'grid_size': 15000, 'step_xy': 6.6671111407422485e-09, 'energy(in eV)': 4000, 'wavelength in m': 3.0996e-10, 'focal_length': 0.006, 'zp_coords': (-50, 50, -50, 50), 'delta': 0.00018332, 'beta': 5.6552e-05}


In [8]:
for var1 in range(6):
    n = num_zones[var1]
    zp =  make_zp_from_rings(n,grid_size)
    for var2 in range(6) :
        if var1+var2 !=0 :
            print(var1,var2)
            max_loc = []
            t = thickness[var2]
            out_dir = '/raid/home/sajid/ZP_simulations/var_width_thickness/4kev/output/'+ str('zones_'+str(n)+'thickness_'+str(t))
            os.mkdir(out_dir)
            print(n,t,grid_size)
            for angle in inputs:
                max_loc.append(tilt(angle,zp,t,parameters,out_dir))
            np.save('max_loc.npy', max_loc) 
          

0 1
1
50 1e-06 15000
claclulating for tilt angle :  -0.918367346939
6.66711114074e-09 3.0996e-10 1e-06
suggested step size : 1.43406797532e-06
number of steps required for propogation through the zone plate : 2
propogator to be used : TF
Fresnel Number : 129066117.779


Propogation through zone Plate...: 100%|██████████| 4/4 [02:15<00:00, 33.87s/it]


Free space propogation after zone Plate...
propogator to be used : IR
Fresnel Number : 5377.75490745
7500 24
tilt image number : -0.918367346939 time taken : 207.79210233688354
7500 24 0.060187461578


/raid/home/sajid/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: tilt_-0.918367346939.tiff is a low contrast image
  warn('%s is a low contrast image' % fname)


0 2
1
50 2e-06 15000


FileNotFoundError: [Errno 2] No such file or directory: '/raid/home/sajid/ZP_simulations/var_width_thickness/4kev/output/zones_50thickness_2e-06'